In [114]:
import re

########################
# Handlers de columnas #
########################

# Sort from lower to higher so that the lowest is 0 and the highest is n
def sortColumnByPriceAverage(X, columnName):
  companies = {}

  for product in X[columnName]:
    companies[product] = (0, 0)

  for i, company in enumerate(X[columnName]):
    price = X["Price"][i]
    val, n = companies[company]
    val += price

    companies[company] = (val, n+1)

  _companies = [ (company, companies[company][0]) for company in companies ]
  sortedCompanies = list(map(lambda t: t[0], sorted(_companies, key=lambda t: t[1])))

  return sortedCompanies

##########
# Screen #
##########

def handle_screen_pixels(row):
  resolution = re.findall("\d+x\d+", row["ScreenResolution"])[0]
  w, h = [ int(_) for _ in resolution.split("x") ] # Ancho y alto

  return w*h

def handle_is_screen_ips(row):
  return "IPS" in row["ScreenResolution"]

def handle_has_touchscreen(row):
  res = row["ScreenResolution"]
  return "Touchscreen" in res or "touchscreen" in res

#######
# CPU #
#######

def handle_cpu_speed(row):
  return float( row["Cpu"].split()[-1][:-3] )

def handle_cpu_brand(row):
  return row["Cpu"].split()[1]

def handle_cpu_antiguedad(row):
  model = row ["Cpu"]
  #display (model)
  
  n = re.findall("(\d{4})", model) # Obtener el numero
  #display (n)

  if "Intel" not in model:
    if ("AMD A" in model):
        amd = (row ["Cpu"].split()[2])
        if (len(amd) != 0):
          #display(amd)
          n = re.findall("(\d{4})", amd)                                                                                    #TERMINAR HAY QUE COGER LAS A5, A6, A7...
          return  "Moderna" if int(n[0].strip()) > 6000 else "Aceptable" if int(n[0].strip()) > 3000 else "Antigua"         # No lo ha encontrado o no lo tiene. Si pongo 0 aumenta. Puede ser overfitting.
        else: return "Error"
  else:
    if (len(n) != 0): 
      return "Moderna" if int(n[0].strip()) > 8000 else "Aceptable" if int(n[0].strip()) > 5000 else "Antigua"
    else:
      return "Error"

#######
# GPU #
#######

def handle_gpu_model(row):
  regex_amd = "AMD (?:FirePro|R\d|Radeon (?:R\d?\w?)?)"
  regex_arm = "ARM Mali"
  regex_intel = "Intel [^\d]*"
  regex_nvidia = "Nvidia (?:(?:GeForce)?\s?G?T?X?|Quadro)+"

  res = re.findall(regex_amd+"|"+regex_arm+"|"+regex_intel+"|"+regex_nvidia, row["Gpu"])

  if res:
    return res[0].strip()
  else:
    print("Model not found", row["Gpu"], res)
    return ""


def handle_gpu_model_number(row):
  model = handle_gpu_model(row)
  model_number = row["Gpu"].replace(model, "").strip() # El numero es lo que se queda al quitarle el modelo

  n = re.findall("\d+", model_number) # Obtener el numero

  if not n:
    return 2 # No lo ha encontrado o no lo tiene. Si pongo 0 aumenta. Puede ser overfitting.
  else:
    return int(n[0].strip())

def handle_gpu_model_gama(row):
  model = handle_gpu_model(row)
  model_number = row["Gpu"].replace(model, "").strip() # El numero es lo que se queda al quitarle el modelo

  n = re.findall("\d+", model_number) # Obtener el numero

  if not n:
    return "Indeterminada" # No lo ha encontrado o no lo tiene. Si pongo 0 aumenta. Puede ser overfitting.
  else:
    #"Muy Alta" if int(n[0].strip()) > 1000 else 
    return "Alta" if int(n[0].strip()) > 700 else "Media" if int(n[0].strip()) > 500 else "Baja"

###########
# Storage #
###########

def parse_memory(mem):
   regex_memoria = "(\d*\.?\d+)(\w+) (SSD|HDD|Flash Storage|Hybrid)" # Regex para identificar la memoria
   return [ x for x in re.findall(regex_memoria, mem) ] # Lista con tuplas de todos los discos encontrados

def handle_storage_capacity(row):
  productos = parse_memory(row["Memory"])
  capacity = sum([
      float(producto[0]) * (1000 if producto[1] == "TB" else 1) for producto in productos
  ])

  return capacity

def handle_has_ssd(row):
  productos = parse_memory(row["Memory"])
  return 1 if any([ producto[2] == "SSD" for producto in productos ]) else 0

######
# OS #
######

def handle_is_os_free(row):
  return row["OpSys"] not in ["No OS", "Linux"] # Cambiar


In [115]:
#@title  { vertical-output: true, form-width: "400px" }
import pandas as pd
import numpy as np
import seaborn as sns
import re
from copy import deepcopy
import statistics as stat
import sklearn.compose
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet
#from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import StandardScaler,PolynomialFeatures,OneHotEncoder,MinMaxScaler,OrdinalEncoder
from sklearn.model_selection import GridSearchCV

# Random forest rands:
# 16 -> 185
# 29487 -> 185.52
# 37057 -> 185.61 / 7
# 37919 -> 185.23 / 6.74

#
# Ideas: Cpu novelty
#


# Ignorar errores de divergencias cuando alpha=0
import warnings
warnings.filterwarnings('ignore')

##########
# Modelo #
##########
class Model:
  def __init__(self, alpha, ratio):
    self.TRAIN_URL = "https://filebin.net/3sw6utbuul0sh225/train.csv"
    self.TEST_URL  = "https://filebin.net/3sw6utbuul0sh225/test.csv"

    self.alpha = alpha
    self.ratio = ratio

    self.X = None
    self.Y = None
    self.X_T = None
    self.Y_T = None

    self.trained_model = None
    self.prediction = None
    self.csv = None

    self.pipe = None # El conjunto de regresores que se aplican

    self.X_original = None
    self.X_final = None

  ###########
  # Dataset #
  ###########
  def load_datasets(self):
    self.X = pd.read_csv(self.TRAIN_URL, sep=",")
    self.Y = pd.read_csv(self.TEST_URL,  sep=",")
    
    # Mantener una copia para echarle un vistazo
    self.X_original = self.X.copy(deep=True)
    self.Y_original = self.X.copy(deep=True)
  
  ##############################
  # Transformacion de columnas #
  ##############################
  # Aplica la funcion f a todos los elementos de una columna
  def map_column(self, columna, f):
    self.X[columna] = [ f(val) for val in self.X[columna] ]
    self.Y[columna] = [ f(val) for val in self.Y[columna] ]
  
  def add_column(self, columna, f):
    self.X[columna] = self.X.apply(lambda row: f(row), axis=1)
    self.Y[columna] = self.Y.apply(lambda row: f(row), axis=1)

  # Modifica los valores de cada columna. Se pueden comentar y no pasa nada.
  def modify_columns(self):
    # Listas auxiliares ordenadas por peso
    sortedCompanies = sortColumnByPriceAverage(self.X, "Company")
    sortedTypeNames = sortColumnByPriceAverage(self.X, "TypeName")
    #sortedOS = ["No OS", "Linux", "Android", "Chrome OS", "Windows 10 S", "Windows 7", "Windows 10", "macOS", "Mac OS X"]

    # New colums
    self.add_column("StorageCapacity", handle_storage_capacity)
    self.add_column("HasSSD", handle_has_ssd)
    self.add_column("ScreenPixels", handle_screen_pixels)
    self.add_column("IsScreenIPS", handle_is_screen_ips)
    self.add_column("HasTouchscreen", handle_has_touchscreen)
    self.add_column("IsOSFree", handle_is_os_free)
    self.add_column("CPUBrand", handle_cpu_brand)
    self.add_column("CPUSpeed", handle_cpu_speed)
    self.add_column("CPUModelAntiguedad", handle_cpu_antiguedad)
    self.add_column("GPUModel", handle_gpu_model)
    self.add_column("GPUModelNumber", handle_gpu_model_number) 
    self.add_column("GPUModelGama", handle_gpu_model_gama)                      # Gama

    self.map_column("Ram",              lambda val: int(val[:-2]))
    self.map_column("Weight",           lambda val: float(val[:-2]))            # Baja un poooooooooooco el error
    self.map_column("Company",          lambda val: sortedCompanies.index(val))
    self.map_column("TypeName",         lambda val: sortedTypeNames.index(val))

    # Another X for consulting and visualization
    self.X_final = self.X.copy(deep=True)\
      .drop(["Product", "OpSys", "Company", "ScreenResolution", "Cpu", "Gpu", "Memory"], axis=1)

  def transform_datasets(self):
    transformersX = [
        ("LaptopId",          "drop",               [0]),
        ("Company",           "passthrough",        [1]), # probar drop
        ("Product",           "drop",               [2]),
        ("TypeName",          "passthrough",        [3]), # probar drop
        ("Inches",            "passthrough",        [4]),
        ("ScreenResolution",  "drop",               [5]),
        ("Cpu",               "drop",               [6]),
        ("Ram",               StandardScaler(),     [7]),
        ("Memory",            "drop",               [8]),
        ("Gpu",               "drop",               [9]),
        ("OpSys",             "drop",               [10]),
        ("Weight",            "passthrough",        [11]),
        ("Price",             "drop",               [12]),

        # New Columns
        ("StorageCapacity",       "passthrough",        [13]),
        ("HasSSD",                OneHotEncoder(),      [14]),
        ("ScreenPixels",          StandardScaler(),     [15]),
        ("IsScreenIPS",           OneHotEncoder(),      [16]),
        ("HasTouchscreen",        OneHotEncoder(),      [17]),
        ("IsOSFree",              "passthrough",        [18]), # TODO: Return boolean and transform
        ("CPUBrand",              OrdinalEncoder(),     [19]),
        ("CPUSpeed",              StandardScaler(),     [20]),
        ("CPUModelAntiguedad",    OrdinalEncoder(),     [21]),
        ("GPUModel",              OrdinalEncoder(),     [22]),
        ("GPUModelNumber",        OrdinalEncoder(),     [23]),
        ("GPUModelGama",          OrdinalEncoder(),     [24])
    ]

    transformersY = deepcopy(transformersX)

    # Normbrar el orden de columnas de 0 a n-1
    transformersY.pop(12) # Eliminar precio
    for i, t in enumerate(transformersY): # Renumerar los indices de las columnas
      transformersY[i][2].pop()
      transformersY[i][2].append(i)

    self.X_T = sklearn.compose.ColumnTransformer(transformers=transformersX).fit_transform(self.X)
    self.Y_T = sklearn.compose.ColumnTransformer(transformers=transformersY).fit_transform(self.Y)

    # StandardScaler. No cambia mucho.
    self.X_T = sklearn.preprocessing.StandardScaler().fit_transform(self.X_T)
    self.Y_T = sklearn.preprocessing.StandardScaler().fit_transform(self.Y_T)
  
  ##########################
  # Regresion y prediccion #
  ##########################
  def set_pipe(self):
    #import random
    #rand = random.randint(0, 1000000000)
    #print(rand)

    pipe = Pipeline([#('poly', PolynomialFeatures(degree=1)),
                     #('standardscaler', StandardScaler()),
                     #('elasticnet',  ElasticNet(alpha=self.alpha, l1_ratio=self.ratio)),
                     #('sgd', SGDRegressor(max_iter=100, random_state=42)),
                     #('extratrees', ExtraTreesRegressor(n_estimators=40)),
                     ('randomforest', RandomForestRegressor(max_depth=50, random_state=37919))
    ])

    self.pipe = pipe

  def apply_regression(self):
    self.trained_model = self.pipe.fit(self.X_T, self.X["Price"])

  def predict(self):
    self.prediction = self.trained_model.predict(self.Y_T)

  #######
  # CSV #
  #######
  def to_csv(self):
    self.solucion = pd.DataFrame(data = {
        "LaptopId" : self.Y["LaptopId"],
        "Price"    : self.prediction
    })
  
  def save_csv(self, name):
    self.solucion.to_csv(name, index=False)

  #################
  # Ejecutar todo #
  #################
  def run(self):
    # Obtener y parsear datasets
    self.load_datasets()
    self.modify_columns()
    self.transform_datasets()

    # Aplicar regresion y hacer una prediccion
    self.set_pipe()
    self.apply_regression()
    self.predict()

    # Convertir a csv
    self.to_csv()
    self.save_csv("solucion.csv")
  
  def print_avgs(self):
    train_avg = sum(self.X["Price"]) / len(self.X["Price"])
    pred_avg  = sum(self.prediction) / len(self.prediction)

    print("Medias\n" + "-" * 100)
    print("Avg training set :", train_avg)
    print("Avg prediction   :", pred_avg)
    print("Diff             :", abs(train_avg - pred_avg), "\n")
    #print("\nScore:", self.trained_model.score()) TODO: FIX

############################
# Clase para probar el MAE #
############################
class ModelTest(Model):
  def __init__(self, alpha, ratio, partition_n):
    # Inicializar padre. Tiene que ser la primera llamada dentro del __init__
    super().__init__(alpha, ratio) 

    self.PARTITION_N = partition_n # Numero de particiones que se van a probar
    self.maes = None               # Array con los maes de cada intento

  def get_maes(self):
    # Prueba n particiones para training y validacion y devuelve el error obtenido en cada intento
    self.maes = cross_val_score(self.pipe, self.X_T, self.X["Price"],
                                cv=self.PARTITION_N, scoring="neg_mean_absolute_error")

  def run(self):
    self.load_datasets()
    self.modify_columns()
    self.transform_datasets()
    self.set_pipe()
    self.get_maes()

  def print_MAE(self):
    print("MAEs    :", self.maes)
    print("MAE avg :", stat.mean(self.maes))
    print("MAE std :", stat.variance(self.maes)**(1/2))

def main():
  es_prueba = True   #@param {type:"boolean"}
  alpha = 0          #@param {type:"raw"}
  ratio = 0          #@param {type:"raw"}
  partition_n = 5    #@param {type:"slider", min:0, max:10,  step:1}

  if es_prueba:
    # Ejecutar para hacer pruebas
    model_test = ModelTest(alpha=alpha, ratio=ratio, partition_n=partition_n)
    model_test.run()
    model_test.print_MAE()

    return model_test

  else:
    # Ejecutar para guardar el resultado
    model = Model(alpha=alpha, ratio=ratio)
  
    model.run()
    model.print_avgs()

    return model

model = main()


MAEs    : [-190.36684078 -179.31895777 -176.53979981 -183.68094158 -186.80608408]
MAE avg : -183.3425248049636
MAE std : 5.565921819095464


In [116]:
model.X_final

,LaptopId,TypeName,Inches,Ram,Weight,Price,StorageCapacity,HasSSD,ScreenPixels,IsScreenIPS,HasTouchscreen,IsOSFree,CPUBrand,CPUSpeed,CPUModelAntiguedad,GPUModel,GPUModelNumber,GPUModelGama
0,981,5,13.3,4,1.20,1195.00,128.0,1,2073600,False,False,True,Core,2.3,Aceptable,Intel HD Graphics,520,Media
1,996,3,13.3,8,1.11,1349.00,256.0,1,2073600,True,True,True,Core,2.7,Aceptable,Intel HD Graphics,620,Media
2,77,5,15.6,8,2.02,855.00,1128.0,1,2073600,False,False,True,Core,1.8,Moderna,Intel UHD Graphics,620,Media
3,103,3,13.3,8,1.38,1119.00,256.0,1,2073600,True,False,True,Core,2.7,Aceptable,Nvidia GeForce,150,Baja
4,946,3,12.5,8,1.36,1472.20,256.0,1,2073600,False,True,True,Core,2.5,Aceptable,Intel HD Graphics,2,Indeterminada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,994,2,13.3,8,1.68,889.00,256.0,1,2073600,False,True,True,Core,2.5,Aceptable,Intel HD Graphics,620,Media
908,615,4,17.3,8,3.00,1168.00,1128.0,1,2073600,False,False,True,Core,2.8,Aceptable,Nvidia GeForce GTX,1050,Alta
909,1063,5,15.6,16,2.32,989.99,2000.0,0,2073600,False,False,True,Core,2.7,Aceptable,AMD Radeon R7,445,Baja
910,96,5,15.6,8,2.20,599.90,1000.0,0,2073600,False,False,False,Core,2.7,Aceptable,AMD Radeon R5,430,Baja


In [117]:
display(model.X_T[:10])

array([[-0.88371329,  0.7293832 , -1.20960244, -0.88933703, -1.26029362,
        -1.03048827, -0.71584762,  0.71584762, -0.05231847,  0.60775766,
        -0.60775766,  0.42044523, -0.42044523,  0.30581887,  0.06389581,
        -0.01815582, -0.57026858, -0.28402534, -0.51212757,  0.88896982],
       [ 0.17119053, -0.86772959, -1.20960244, -0.08109052, -1.39490179,
        -0.75348683, -0.71584762,  0.71584762, -0.05231847, -1.64539268,
         1.64539268, -2.37843106,  2.37843106,  0.30581887,  0.06389581,
         0.77790695, -0.57026858, -0.28402534,  0.11915693,  0.88896982],
       [ 0.87445975,  0.7293832 ,  0.40762114, -0.08109052, -0.0338636 ,
         1.13358551, -0.71584762,  0.71584762, -0.05231847,  0.60775766,
        -0.60775766,  0.42044523, -0.42044523,  0.30581887,  0.06389581,
        -1.01323428,  1.97500848,  0.5409145 ,  0.11915693,  0.88896982],
       [ 0.17119053, -0.86772959, -1.20960244, -0.08109052, -0.99107727,
        -0.75348683, -0.71584762,  0.71584762, -

In [118]:
def sortCompaniesByPriceAverage(model):
  companies = {}

  for product in model.X["Company"]:
    companies[product] = (0, 0)

  for i, company in enumerate(model.X["Company"]):
    price = model.X["Price"][i]
    val, n = companies[company]
    val += price

    companies[company] = (val, n+1)

  _companies = [ (company, companies[company][0]) for company in companies ]
  sortedCompanies = list(map(lambda t: t[0], sorted(_companies, key=lambda t: -t[1])))

  return sortedCompanies

sortCompaniesByPriceAverage(model)


[18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [119]:
model.X

,LaptopId,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,...,ScreenPixels,IsScreenIPS,HasTouchscreen,IsOSFree,CPUBrand,CPUSpeed,CPUModelAntiguedad,GPUModel,GPUModelNumber,GPUModelGama
0,981,13,Portege Z30-C-1CV,5,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4,128GB SSD,Intel HD Graphics 520,...,2073600,False,False,True,Core,2.3,Aceptable,Intel HD Graphics,520,Media
1,996,16,Spectre 13-V111dx,3,13.3,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,8,256GB SSD,Intel HD Graphics 620,...,2073600,True,True,True,Core,2.7,Aceptable,Intel HD Graphics,620,Media
2,77,18,Inspiron 5570,5,15.6,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8,128GB SSD + 1TB HDD,Intel UHD Graphics 620,...,2073600,False,False,True,Core,1.8,Moderna,Intel UHD Graphics,620,Media
3,103,16,Envy 13-ad009n,3,13.3,IPS Panel Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8,256GB SSD,Nvidia GeForce MX150,...,2073600,True,False,True,Core,2.7,Aceptable,Nvidia GeForce,150,Baja
4,946,18,Latitude 7280,3,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics,...,2073600,False,True,True,Core,2.5,Aceptable,Intel HD Graphics,2,Indeterminada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,994,18,Inspiron 5378,2,13.3,Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,...,2073600,False,True,True,Core,2.5,Aceptable,Intel HD Graphics,620,Media
908,615,15,FX753VD-GC007T (i7-7700HQ/8GB/1TB,4,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8,128GB SSD + 1TB HDD,Nvidia GeForce GTX 1050,...,2073600,False,False,True,Core,2.8,Aceptable,Nvidia GeForce GTX,1050,Alta
909,1063,18,Inspiron 5567,5,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,16,2TB HDD,AMD Radeon R7 M445,...,2073600,False,False,True,Core,2.7,Aceptable,AMD Radeon R7,445,Baja
910,96,18,Inspiron 3567,5,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8,1TB HDD,AMD Radeon R5 M430,...,2073600,False,False,False,Core,2.7,Aceptable,AMD Radeon R5,430,Baja


In [120]:
set(model.X_original["Gpu"])
#set(model.X_original["Cpu"])

{'AMD FirePro W4190M ',
 'AMD FirePro W5130M',
 'AMD R4 Graphics',
 'AMD Radeon 520',
 'AMD Radeon 530',
 'AMD Radeon 540',
 'AMD Radeon R2',
 'AMD Radeon R2 Graphics',
 'AMD Radeon R3',
 'AMD Radeon R4',
 'AMD Radeon R4 Graphics',
 'AMD Radeon R5',
 'AMD Radeon R5 430',
 'AMD Radeon R5 520',
 'AMD Radeon R5 M330',
 'AMD Radeon R5 M420',
 'AMD Radeon R5 M420X',
 'AMD Radeon R5 M430',
 'AMD Radeon R7',
 'AMD Radeon R7 Graphics',
 'AMD Radeon R7 M440',
 'AMD Radeon R7 M445',
 'AMD Radeon R7 M460',
 'AMD Radeon R7 M465',
 'AMD Radeon R9 M385',
 'AMD Radeon RX 540',
 'AMD Radeon RX 550',
 'AMD Radeon RX 560',
 'AMD Radeon RX 580',
 'ARM Mali T860 MP4',
 'Intel Graphics 620',
 'Intel HD Graphics',
 'Intel HD Graphics 400',
 'Intel HD Graphics 405',
 'Intel HD Graphics 500',
 'Intel HD Graphics 505',
 'Intel HD Graphics 510',
 'Intel HD Graphics 515',
 'Intel HD Graphics 520',
 'Intel HD Graphics 5300',
 'Intel HD Graphics 540',
 'Intel HD Graphics 6000',
 'Intel HD Graphics 615',
 'Intel HD

In [121]:
def precioMemoria(producto):
  m1 = float(producto[0])
  m2 = ["_", "GB", "TB"].index(producto[1])
  m3 = ["_", "HDD", "SSD", "Hybrid", "Flash Storage"].index(producto[2])
  
  return m1*m2*m3

memory = set(model.X["Memory"])
regex = "(\d*\.?\d+)(\w+) (SSD|HDD|Flash Storage|Hybrid)"

memoryFound = [ sum([ precioMemoria(x) for x in re.findall(regex, mem)]) for mem in memory ]
memoryFound
# x TB HDD
# x GB HDD
# x GB SSD
# x TB Hybrid
# x GB Flash Storage
# y + z
# 

[500.0,
 4.0,
 258.0,
 1524.0,
 128.0,
 6.0,
 6.0,
 128.0,
 1024.0,
 514.0,
 4.0,
 32.0,
 128.0,
 512.0,
 480.0,
 256.0,
 518.0,
 64.0,
 1026.0,
 1028.0,
 64.0,
 2.0,
 1012.0,
 2.0,
 256.0,
 512.0,
 360.0,
 2048.0,
 260.0,
 2048.0,
 516.0,
 1024.0,
 1024.0]

In [122]:
set(model.X["Cpu"])
#1x2
# 4K Ultra HD / Touchscreen 1x2
# 

{'AMD A10-Series 9600P 2.4GHz',
 'AMD A10-Series 9620P 2.5GHz',
 'AMD A10-Series A10-9620P 2.5GHz',
 'AMD A12-Series 9700P 2.5GHz',
 'AMD A12-Series 9720P 2.7GHz',
 'AMD A12-Series 9720P 3.6GHz',
 'AMD A4-Series 7210 2.2GHz',
 'AMD A6-Series 7310 2GHz',
 'AMD A6-Series 9220 2.5GHz',
 'AMD A6-Series 9220 2.9GHz',
 'AMD A6-Series A6-9220 2.5GHz',
 'AMD A8-Series 7410 2.2GHz',
 'AMD A9-Series 9410 2.9GHz',
 'AMD A9-Series 9420 3GHz',
 'AMD A9-Series A9-9420 3GHz',
 'AMD E-Series 6110 1.5GHz',
 'AMD E-Series 7110 1.8GHz',
 'AMD E-Series 9000 2.2GHz',
 'AMD E-Series 9000e 1.5GHz',
 'AMD E-Series E2-6110 1.5GHz',
 'AMD E-Series E2-9000 2.2GHz',
 'AMD E-Series E2-9000e 1.5GHz',
 'AMD FX 8800P 2.1GHz',
 'AMD FX 9830P 3GHz',
 'AMD Ryzen 1600 3.2GHz',
 'AMD Ryzen 1700 3GHz',
 'Intel Atom X5-Z8350 1.44GHz',
 'Intel Atom x5-Z8350 1.44GHz',
 'Intel Atom x5-Z8550 1.44GHz',
 'Intel Celeron Dual Core 3205U 1.5GHz',
 'Intel Celeron Dual Core 3855U 1.6GHz',
 'Intel Celeron Dual Core N3050 1.6GHz',
 'Int

In [125]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Relplot
# Scatterplot
# Density plot
# Characteristic line plot

class Draw:
  def __init__(self, model):
    self.model = model
    self.labels = ["Company", "Product", "TypeName", "Inches", "ScreenResolution", "Cpu", "Ram", "Memory", "Gpu", "OpSys", "Weight"]

  def get_grid(self, cols, rows, h, w):
      return plt.subplots(rows, cols, figsize=(w, h))

  def histogram(self, labels):
    fig0, axs0 = plt.subplots(ncols=4)
    fig0.set_figwidth(30)
    fig0.set_figheight(5)

    for i, label in enumerate(labels):
      aux = sns.histplot(self.model.X[label], ax=axs0[i])

      aux.set_xticklabels(aux.get_xticklabels(), rotation = 80, horizontalalignment = 'right')

    #plt.show()

  def countplot(self, labels):
    fig0, axs0 = plt.subplots(ncols=4)
    fig0.set_figwidth(30)
    fig0.set_figheight(5)

    for i, label in enumerate(labels):
      aux = px.countplot(self.model.X[label], ax=axs0[i])

      #aux0.set_xticklabels(aux0.get_xticklabels(), rotation = 85, horizontalalignment = 'right')

      #aux.set(xticklabels=label)
      #aux.set(xlabel=None)
  
  def kde(self, labels):
    # El numero de caracteristicas que tienen tipo numerico, para decidir las dimensiones
    n_of_numerical_features = len([ True for f in self.model.X if type(self.model.X[f][0]) != str ])
    
    fig, axs = self.get_grid(rows=n_of_numerical_features//4, cols=4, h=10, w=25)
    fig.delaxes(axs[1][2]) # Elimnar plots vacios
    fig.delaxes(axs[1][3])
    
    i = 0
    for label in labels:
      if type(self.model.X[label][0]) != str:
        sns.kdeplot(self.model.X[label], ax=axs[i//4, i%4])
        i += 1
  
  def relplot(self, labels, hue=None):
    fig, axs = self.get_grid(rows=3, cols=4, h=15, w=30)
    fig.delaxes(axs[2][3]) # Elimnar plots vacios

    for i, label in enumerate(labels):
      s = sns.scatterplot(data=self.model.X, x=label, y="Price", ax=axs[i//4, i%4], hue=hue)

      # Si es una feature con strings, quitar los ticks porque si no no se ve nada
      if type(self.model.X[label][0]) == str:
        s.set(xticklabels=[])  
        s.tick_params(bottom=False)

  def run(self):
    #self.countplot(["ScreenResolution", "Cpu", "Gpu", "Weight"])
    #self.histogram(["ScreenResolution", "Cpu", "Gpu", "Weight"])
    #self.kde(self.labels)
    self.relplot(self.labels)
    
    #plt.show()

  
draw = Draw(model)
draw.run()